# 使用tensorflow

tensorflow是一个强大的用于数值计算的开源软件，尤其适用于大型机器学习。
基本原则很简单：首先定义要执行的计算图， 接着tensorflow载入图，用优化过的c++代码执行。

最重要的是，可以将图形分割成多个块，并在多个cpu或gpu之间并行运行它们。也支持分布式计算，因此，你可以通过在数百台服务器上拆分计算，在合理的时间内训练庞大的神经网络。

In [96]:
# 准备工作
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

import numpy as np
import os

# 控制随机种子，使结果一样
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = ".."
CHAPTER_ID = "tensorflow"


def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

# 创建和运行数据流图

In [97]:
import tensorflow as tf

reset_graph()
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x * x * y + y + 2

注意， 上述代码并没有执行任何计算， 即使看起来像是执行了很多计算一样。甚至呆变量也没有初始化。要执行计算，需要创建tensorflow session， 并使用它来初始化和变量和执行计算。
Tensorflow session负责将操作放置到诸如cpu和gpu之类的设备上，并运行它们。如下执行：

In [98]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [99]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
result

42

In [100]:
sess.close()

In [101]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval() #eval:串演算指令

In [102]:
result

42

In [103]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()

In [104]:
result

42

In [105]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
init.run()
result = f.eval()
result

/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


42

In [106]:
sess.close()

tensorflow程序铭心分为两部分：第一部分创建计算图，第二部分执行计算。创建过程表示创建ML模型和需要的计算过程， 执行过程通常循环计算训练的每一步，逐渐改进模型的参数。 

# 组织 数据流图

In [107]:
reset_graph()

x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [108]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
x2.graph is graph

True

In [109]:
x2.graph is tf.get_default_graph()

False

In [110]:
# 节点的声明周期
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())  #10
    print(z.eval())  #15

10
15


In [111]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15



# 线性回归


tensorflow operations(ops) 可以取任意数量的输入并产生任意数量的输出。

## 使用标准方程

In [112]:

import numpy as np
from sklearn.datasets import fetch_california_housing

reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [113]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

## 与使用numpy计算对比

In [114]:
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
theta_numpy

array([[-3.69419202e+01],
       [ 4.36693293e-01],
       [ 9.43577803e-03],
       [-1.07322041e-01],
       [ 6.45065694e-01],
       [-3.97638942e-06],
       [-3.78654266e-03],
       [-4.21314378e-01],
       [-4.34513755e-01]])

## 与使用sklearn对比

In [115]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing_data_plus_bias, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 0.00000000e+00]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


## 使用梯度下降


梯度下降首先需要进行特征缩放，可以使用tf，不过目前还是使用sklearn。

In [122]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bais = np.c_[np.ones((m, 1)), scaled_housing_data]

In [126]:
print(scaled_housing_data_plus_bais.mean(axis=0))
print(scaled_housing_data_plus_bais.mean(axis=1))
print(scaled_housing_data_plus_bais.mean())
print(scaled_housing_data_plus_bais.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


### 一般方法计算梯度

In [133]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bais, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")

y_pred = tf.matmul(X, theta, name="predications")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2 / m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE =  9.161543
Epoch 100 MSE =  0.71450067
Epoch 200 MSE =  0.5667049
Epoch 300 MSE =  0.5555719
Epoch 400 MSE =  0.5488112
Epoch 500 MSE =  0.5436362
Epoch 600 MSE =  0.5396294
Epoch 700 MSE =  0.53650916
Epoch 800 MSE =  0.5340678
Epoch 900 MSE =  0.5321474


In [134]:
best_theta

array([[ 2.0685523 ],
       [ 0.8874027 ],
       [ 0.14401656],
       [-0.34770885],
       [ 0.36178368],
       [ 0.00393811],
       [-0.04269556],
       [-0.66145283],
       [-0.6375278 ]], dtype=float32)

### 使用autodiff
与上述过程一样，出了 gradients=... 这一行

注意：上述过程运行的很好，但是需要从损失函数中推导出梯度。在线性回归中，求偏导数相对容易。但是如果是执行深度神经网络，这个过程会变的困难。虽然可以计算出函数的偏导数，但是得到的代码不一定有效。  


In [136]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bais, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [137]:
gradients = tf.gradients(mse, [theta])[0]

In [138]:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 9.161543
Epoch 100 MSE = 0.7145006
Epoch 200 MSE = 0.566705
Epoch 300 MSE = 0.5555719
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.5436362
Epoch 600 MSE = 0.5396294
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.5340678
Epoch 900 MSE = 0.5321474
Best theta:
[[ 2.0685525 ]
 [ 0.8874027 ]
 [ 0.14401658]
 [-0.34770882]
 [ 0.36178368]
 [ 0.00393811]
 [-0.04269556]
 [-0.6614528 ]
 [-0.6375277 ]]


##### 如何求出如下关于a，b函数的偏导数？

In [145]:
def my_func(a, b):
    z = 0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

In [146]:
my_func(0.2, 0.3)

-0.21253923284754916

## autodiff
tensorflow 的autodiff 特征可以自动并且有效的计算梯度，如下代码所示：
gradients() 函数的将 一个op 和一个变量列表，根据每个变量执行一组列表ops操作， 计算op的梯度。 因此下面代码，gradients 将根据 $\theta$ 和mse 操作 计算梯度向量。

In [149]:
reset_graph()

a = tf.Variable(0.2, name="a")
b = tf.Variable(0.3, name="b")
z = tf.Variable(0.0, name="z0")

for i in range(100):
    z = a * tf.cos(z + i) + z * tf.sin(b - i)

grads = tf.gradients(z, [a, b])
init = tf.global_variables_initializer()

###### 下面计算函数在$a=0.2$ 和 $b=0.3$， 并且关于a, b点的偏导数：

In [150]:
with tf.Session() as sess:
    init.run()
    print(z.eval())
    print(sess.run(grads))

-0.21253741
[-1.1388494, 0.19671395]


## 使用梯度下降优化